# Senpy in 10 minutes

This tutorial assumes you have completed the [1 minute tutorial](Quickstart-1minute.ipynb) and you know how to access the API.

It covers:

* Annotating entiment with a sentiment analysis service
* Annotating emotions
* Interoperability of services
* Using other semantic formats (RDF)

## Requirements

Once again, we will use the demo server at http://senpy.gsi.upm.es.
This time, we will use the requests library.

We will use a variable for our endpoint, so you can try this examples on a different senpy instance:

In [1]:
endpoint = 'http://senpy.gsi.upm.es/api'

We will also add a function to get syntax highlighting for the JSON-LD/Turtle results:

In [2]:
try:
    from IPython.display import Code
    def pretty(txt, language='json-ld'):
        return Code(txt, language=language)
except ImportError:
    def pretty(txt, **kwargs):
        print(txt)

## Sentiment Analysis of Text

To start, let us analyse the sentiment in the following sentence: *senpy is awesome*.

For now, we will use the [sentiment140](http://www.sentiment140.com/) service, through the sentiment140 plugin.
We will later cover how to use a different service.


In [4]:
import requests
res = requests.get(f'{endpoint}/sentiment140',
                   params={"input": "Senpy is awesome",})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudDE0MD9pbnB1dD1TZW5weStpcythd2Vzb21lIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [
        {
          "@type": "Sentiment",
          "marl:hasPolarity": "marl:Positive",
          "prov:wasGeneratedBy": "prefix:Analysis_1554396807.629645"
        }
      ],
      "nif:isString": "Senpy is awesome",
      "onyx:hasEmotionSet": []
    }
  ]
}

Senpy services always return an object of type `senpy:Results`, with a list of entries.
You can think of an entry as a self-contained textual context (`nif:Context` and `senpy:Entry`).
Entries can be as short as a sentence, or as long as a news article.

Each entry has a `nif:isString` property that contains the original text of the entry, and several other properties that are provided by the plugins.

For instance, sentiment annotations are provided through `marl:hasOpinion`.

The annotations are semantic.
We can ask Senpy for the expanded JSON-LD output to reveal the full URIs of each property and entity:

In [13]:
import requests
res = requests.get(f'{endpoint}/sentiment140',
                   params={"input": "Senpy is awesome",
                           "expanded": True})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudDE0MD9pbnB1dD1TZW5weStpcythd2Vzb21lJmV4cGFuZGVkPVRydWUj",
  "@type": [
    "http://www.gsi.upm.es/onto/senpy/ns#Results"
  ],
  "http://www.w3.org/ns/prov#used": [
    {
      "@id": "http://senpy.invalid/",
      "@type": [
        "http://www.gsi.upm.es/onto/senpy/ns#Entry"
      ],
      "http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#isString": [
        {
          "@value": "Senpy is awesome"
        }
      ],
      "http://www.gsi.dit.upm.es/ontologies/marl/ns#hasOpinion": [
        {
          "@type": [
            "http://www.gsi.upm.es/onto/senpy/ns#Sentiment"
          ],
          "http://www.gsi.dit.upm.es/ontologies/marl/ns#hasPolarity": [
            {
              "@value": "marl:Positive"
            }
          ],
          "http://www.w3.org/ns/prov#wasGeneratedBy": [
            {
              "@id": "http://senpy.invalid/Analysis_1554389672.269198"
            }
          ]
        }
      ],
      "http://www.gsi.dit.upm.es/ontologies/onyx/ns#hasEmotionSet": []
    }
  ]
}

## Service interoperability

All senpy plugins use the same API, which makes moving from one service to another a breeze.

Let us go back to our simple example, which uses the `sentiment140` service.

In [14]:
import requests
res = requests.get(f'{endpoint}/sentiment140',
                   params={"input": "Senpy is awesome"})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudDE0MD9pbnB1dD1TZW5weStpcythd2Vzb21lIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [
        {
          "@type": "Sentiment",
          "marl:hasPolarity": "marl:Positive",
          "prov:wasGeneratedBy": "prefix:Analysis_1554389675.2303865"
        }
      ],
      "nif:isString": "Senpy is awesome",
      "onyx:hasEmotionSet": []
    }
  ]
}

If we wanted to use a different service (e.g. the `sentiment-basic` plugin), we can do it just by changing the URL of the service:

In [15]:
import requests
res = requests.get(f'{endpoint}/sentiment-basic',
                   params={"input": "Senpy is awesome"})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL3NlbnRpbWVudC1iYXNpYz9pbnB1dD1TZW5weStpcythd2Vzb21lIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [
        {
          "@type": "Sentiment",
          "marl:hasPolarity": "marl:Neutral",
          "prov:wasGeneratedBy": "prefix:Analysis_1554389675.9911203"
        }
      ],
      "nif:isString": "Senpy is awesome",
      "onyx:hasEmotionSet": []
    }
  ]
}

As you can see, the structure and annotation schema of the response is the same.
This makes it very easy to compare and migrate to different services.

Service interoperability is not only useful for users.
It is also key for other features such as [automated evaluation](#Evaluation).
This is a compelling reason to adapt existing services to use the Senpy API.
In fact, the `sentiment140` senpy service is proxy to the public [Sentiment 140 service](http://www.sentiment140.com/).

## Emotion analysis


Senpy uses the `onyx` vocabulary to represent emotions, which incorporates the notion of `EmotionSet`'s, an emotion that is composed of several emotions.
In a nutshell, an `Entry` is linked to one or more `EmotionSet`, which in turn is made up of one or more `Emotion`.

Let's illustrate it with an example, using the `emotion-depechemood` plugin.

In [16]:
res = requests.get(f'{endpoint}/emotion-depechemood',
                   params={"input": "Senpy is a wonderful that service"})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tZGVwZWNoZW1vb2Q_aW5wdXQ9U2VucHkraXMrYSt3b25kZXJmdWwrdGhhdCtzZXJ2aWNlIw%3D%3D",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful that service",
      "onyx:hasEmotionSet": [
        {
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:negative-fear",
              "onyx:hasEmotionIntensity": 0.06258366271018097
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:amusement",
              "onyx:hasEmotionIntensity": 0.15784834034155437
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:anger",
              "onyx:hasEmotionIntensity": 0.08728815135373413
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:annoyance",
              "onyx:hasEmotionIntensity": 0.12184635680460143
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:indifference",
              "onyx:hasEmotionIntensity": 0.1374081151031531
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:joy",
              "onyx:hasEmotionIntensity": 0.12267040802346799
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:awe",
              "onyx:hasEmotionIntensity": 0.21085262130713067
            },
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:sadness",
              "onyx:hasEmotionIntensity": 0.09950234435617733
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1554389679.0535374"
        }
      ]
    }
  ]
}

As you have probably noticed, there are several emotions in this result, each with a different intensity.

We can also tell senpy to only return the emotion with the maximum intensity using the `maxemotion` parameter:

In [17]:
res = requests.get(f'{endpoint}/emotion-depechemood',
                   params={"input": "Senpy is a wonderful service",
                           "maxemotion": True})
pretty(res.text)

{
  "@context": "http://senpy.gsi.upm.es/api/contexts/YXBpL2Vtb3Rpb24tZGVwZWNoZW1vb2Q_aW5wdXQ9U2VucHkraXMrYSt3b25kZXJmdWwrc2VydmljZSZtYXhlbW90aW9uPVRydWUj",
  "@type": "Results",
  "entries": [
    {
      "@id": "prefix:",
      "@type": "Entry",
      "marl:hasOpinion": [],
      "nif:isString": "Senpy is a wonderful service",
      "onyx:hasEmotionSet": [
        {
          "@type": "EmotionSet",
          "onyx:hasEmotion": [
            {
              "@type": "Emotion",
              "onyx:hasEmotionCategory": "wna:awe",
              "onyx:hasEmotionIntensity": 0.21085262130713067
            }
          ],
          "prov:wasGeneratedBy": "prefix:Analysis_1554389681.1941268"
        }
      ]
    }
  ]
}

## Other output formats

Senpy supports several semantic formats, like turtle and xml-RDF.
You can select the format of the output with the `outformat` parameter:

In [18]:
res = requests.get(f'{endpoint}/sentiment140',
                   params={"input": "Senpy is the best framework for semantic sentiment analysis, and very easy to use",
                            "outformat": "turtle"})
pretty(res.text, language='turtle')

@prefix : <http://www.gsi.upm.es/onto/senpy/ns#> .
@prefix dc: <http://dublincore.org/2012/06/14/dcelements#> .
@prefix emoml: <http://www.gsi.dit.upm.es/ontologies/onyx/vocabularies/emotionml/ns#> .
@prefix endpoint: <http://senpy.gsi.upm.es/api/> .
@prefix fam: <http://vocab.fusepool.info/fam#> .
@prefix marl: <http://www.gsi.dit.upm.es/ontologies/marl/ns#> .
@prefix nif: <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#> .
@prefix onyx: <http://www.gsi.dit.upm.es/ontologies/onyx/ns#> .
@prefix prefix: <http://senpy.invalid/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix senpy: <http://www.gsi.upm.es/onto/senpy/ns#> .
@prefix wna: <http://www.gsi.dit.upm.es/ontologies/wnaffect/ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

prefix: a senpy:Entry ;
    nif:isString "Senpy is the best framework for semantic sentiment analysis, and very easy to use" ;
    marl:hasOpinion [ a senpy:Sentiment ;
            marl:hasPolarity "marl:Positive" ;
            prov:wasGeneratedBy prefix:Analysis_1554389682.350598 ] .

[] a senpy:Results ;
    prov:used prefix: .